In [1]:
print("Notebook is working")

Notebook is working


## Imports

In [2]:
import requests
from datetime import datetime, time, timezone, date as dt_date

## Stadium and game information

In [40]:
stadiums = {
    "Truist Park": {"latitude": 33.8908, "longitude": -84.4678, "team_id": 144},
    "Sutter Health Park": {"latitude": 38.58018, "longitude": -121.51384, "team_id": None},
    "Busch Stadium": {"latitude": 38.62278, "longitude": -90.19333, "team_id": None}
}

stadium_name = "Busch Stadium"
latitude = stadiums[stadium_name]["latitude"]
longitude = stadiums[stadium_name]["longitude"]
team_id = stadiums[stadium_name]["team_id"]
date = dt_date.today().isoformat()


In [3]:
mlb_stadiums = {
    "Braves": {
        "stadium": "Truist Park",
        "latitude": 33.8908,
        "longitude": -84.4678,
        "outfield_direction": 45,  # NE
        "team_id": 144
    },
    "Athletics": {
        "stadium": "Sutter Health Park",
        "latitude": 38.58018,
        "longitude": -121.51384,
        "outfield_direction": 65,  # ENE
        "team_id": 133
    },
    "Phillies": {
        "stadium": "Citizens Bank Park",
        "latitude": 39.9057,
        "longitude": -75.1665,
        "outfield_direction": 25,  # NE
        "team_id": 143
    },
    "Cardinals": {
        "stadium": "Busch Stadium",
        "latitude": 38.62278,
        "longitude": -90.19333,
        "outfield_direction": 80,  # ENE
        "team_id": 138
    }
}


In [4]:
selected_team = "Athletics"  # Change this to test other teams

team_info = mlb_stadiums[selected_team]
stadium_name = team_info["stadium"]
latitude = team_info["latitude"]
longitude = team_info["longitude"]
outfield_direction_deg = team_info["outfield_direction"]
team_id = team_info["team_id"]
date = dt_date.today().isoformat()


In [6]:
mlb_url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={date}&teamId={team_id}"
mlb_response = requests.get(mlb_url)
mlb_data = mlb_response.json()

games = mlb_data.get("dates", [])
if games:
    game = games[0]["games"][0]
    utc_start = game["gameDate"]
    print("Raw UTC start time:", utc_start)

    utc_dt = datetime.strptime(utc_start, "%Y-%m-%dT%H:%M:%SZ")
    local_hour = (utc_dt.hour - 4) % 24  # Adjust if needed
    game_hour = local_hour
    print("Local game hour:", game_hour)
else:
    print("No game scheduled today.")
    game_hour = None


Raw UTC start time: 2025-07-13T02:05:00Z
Local game hour: 22


In [8]:
# Outfield wind directions by stadium (degrees wind blows "out")
outfield_directions = {
    "Truist Park": 45,              # Braves - Outfield faces N
    "Citizens Bank Park": 25,        # Phillies - Outfield faces N
    "Sutter Health Park": 65,        # Athletics - Outfield faces NE
    "Busch Stadium": 80              # Cardinals - Outfield faces E
    #add more later
}


In [9]:
if games:
    game = games[0]["games"][0]  # First Braves game today
    stadium_name = game["venue"]["name"]   # Get the stadium name
    print("Stadium:", stadium_name)

    outfield_direction_deg = outfield_directions.get(stadium_name)

    if outfield_direction_deg is None:
        print(f"⚠️ No outfield direction known for {stadium_name}. Wind effect analysis will be skipped.")


    utc_start = game["gameDate"]  # e.g. "2025-07-01T23:20:00Z"
    print("Raw UTC start time:", utc_start)
else:
    print("No Braves game scheduled today.")


Stadium: Sutter Health Park
Raw UTC start time: 2025-07-13T02:05:00Z


In [10]:
# Convert UTC time to your local time zone
dt_utc = datetime.fromisoformat(utc_start.replace("Z", "+00:00"))
dt_local = dt_utc.astimezone(tz=None)  # Auto-detects your system's local time
game_hour = dt_local.hour

print("Local start time:", dt_local)
print("Start hour only (for filtering):", game_hour)


Local start time: 2025-07-12 22:05:00-04:00
Start hour only (for filtering): 22


In [11]:
# Open-Meteo API
weather_url = "https://api.open-meteo.com/v1/forecast"
weather_params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": date,
    "end_date": date,
    "hourly": "winddirection_10m,windspeed_10m",
    "timezone": "America/New_York"
}

weather_resp = requests.get(weather_url, params=weather_params)
weather_data = weather_resp.json()


In [12]:
def filter_for_game_time(data, start_hour, end_hour):
    hourly = data["hourly"]
    times = hourly["time"]
    directions = hourly["winddirection_10m"]
    speeds_kmh = hourly["windspeed_10m"]

    filtered = []

    for i, t_str in enumerate(times):
        dt = datetime.strptime(t_str, "%Y-%m-%dT%H:%M")
        if start_hour <= dt.hour <= end_hour:
            filtered.append({
                "time": t_str,
                "direction_deg": directions[i],
                "speed_kmh": speeds_kmh[i],
                "speed_mph": round(speeds_kmh[i] * 0.621371, 2)
            })

    return filtered


In [14]:
def degrees_to_cardinal(degrees):
    dirs = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    ix = round(degrees / 45) % 8
    return dirs[ix]

In [15]:
if game_hour is not None:
    filtered_wind = filter_for_game_time(weather_data, start_hour=game_hour, end_hour=game_hour+3)

    for entry in filtered_wind:
        cardinal = degrees_to_cardinal(entry["direction_deg"])

        # Only calculate wind effect if outfield direction is known
        if outfield_direction_deg is not None:
            # Calculate angle difference between wind direction and outfield
            wind_blowing_toward = (entry["direction_deg"] + 180) % 360
            angle_diff = abs(wind_blowing_toward - outfield_direction_deg)
            angle_diff = min(angle_diff, 360 - angle_diff)

            if angle_diff <= 45:
                effect = "Blowing out"
            elif angle_diff >= 135:
                effect = "Blowing in"
            else:
                effect = "Cross-wind"


            effect_text = f", Effect: {effect}"
        else:
            effect_text = ""

        print(f"Time: {entry['time']}, Direction: {entry['direction_deg']}° ({cardinal}), "
              f"Speed: {entry['speed_kmh']} km/h ({entry['speed_mph']} mph){effect_text}")

else:
    print("No game today — no weather filtering.")


Time: 2025-07-12T22:00, Direction: 231° (SW), Speed: 20.2 km/h (12.55 mph), Effect: Blowing out
Time: 2025-07-12T23:00, Direction: 232° (SW), Speed: 18.2 km/h (11.31 mph), Effect: Blowing out
